In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 1. Load Data
def load_data(file_path):
    """
    Load data from a CSV file into a pandas DataFrame.
    """
    data = pd.read_csv(file_path)
    return data

# 2. Create a user-item matrix
def create_user_item_matrix(data):
    """
    Create a user-item matrix where rows are users and columns are items.
    """
    user_item_matrix = data.pivot_table(index='user_id', columns='item_id', values='rating')
    return user_item_matrix.fillna(0)  # Replace NaN with 0 (no interaction)

# 3. Compute Item Similarity Matrix
def compute_item_similarity(user_item_matrix):
    """
    Compute cosine similarity between items using the user-item matrix.
    """
    item_similarity = cosine_similarity(user_item_matrix.T)  # Similarity between items
    item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)
    return item_similarity_df

# 4. Generate Predictions
def generate_predictions(user_item_matrix, item_similarity_df, user_id):
    """
    Generate item recommendations for a given user based on item similarity.
    """
    user_ratings = user_item_matrix.loc[user_id]
    similar_items = item_similarity_df.dot(user_ratings) / item_similarity_df.sum(axis=1)
    
    # Filter out items already rated by the user
    recommendations = similar_items[~user_ratings.index.isin(user_ratings[user_ratings > 0].index)]
    
    # Sort recommendations based on predicted ratings
    recommendations = recommendations.sort_values(ascending=False)
    
    return recommendations

# 5. Main function to run the recommendation system
def main(file_path, user_id):
    data = load_data(file_path)
    print(f"data=\n{data}");print()
    
    user_item_matrix = create_user_item_matrix(data)
    print(f"user_item_matrix=\n{user_item_matrix}");print()
    
    item_similarity_df = compute_item_similarity(user_item_matrix)
    print(f"item_similarity_df=\n{item_similarity_df}");print()
    
    recommendations = generate_predictions(user_item_matrix, item_similarity_df, user_id)
    
    return recommendations


In [12]:
if __name__ == "__main__":
    # Example file path (replace with actual CSV path)
    file_path = 'ratings.csv'
    
    user_id = 1
    recommendations = main(file_path, user_id)
    print(f"Recommendations for User {user_id}:")
    print(recommendations.head(10))


data=
    user_id  item_id  rating
0         1      101       5
1         1      102       3
2         1      103       2
3         2      101       4
4         2      102       3
5         2      104       5
6         3      101       2
7         3      103       5
8         3      104       4
9         4      102       4
10        4      103       5
11        4      104       1

user_item_matrix=
item_id  101  102  103  104
user_id                    
1        5.0  3.0  2.0  0.0
2        4.0  3.0  0.0  5.0
3        2.0  0.0  5.0  4.0
4        0.0  4.0  5.0  1.0

item_similarity_df=
item_id       101       102       103       104
item_id                                        
101      1.000000  0.690268  0.405720  0.644061
102      0.690268  1.000000  0.606788  0.502793
103      0.405720  0.606788  1.000000  0.524951
104      0.644061  0.502793  0.524951  1.000000

Recommendations for User 1:
item_id
104    2.162803
dtype: float64
